I'm using ACG because I can define a boundary, which is useful when determining what magnitude of attack can be detected

In [1]:
AGENT_NAME = r'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2'
SAVE_DIR = 'intermittant acg/'
ATK_START = 8000
ATK_LEN = 24
ATK_NAME = f'untargeted_acg_step{ATK_START}_len_{ATK_LEN}'

In [30]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion import AutoConjugateGradient as ACG

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from copy import deepcopy
import json

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)
agent = PPO.load(path=f"{AGENT_NAME}",
                 print_system_info=True)
env = utils.make_discrete_env(schema=schema,  
                        action_bins=agent.action_space[0].n,
                        seed=42)

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.0
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.0
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0



In [4]:
cols = env.observation_names

In [5]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features

In [7]:
init = 50
iter = int(500/init)
kwargs = dict(
    loss_type='difference_logits_ratio', 
    #batch_size=1,
    nb_random_init=init, #5, lower values speed crafting
    max_iter=iter, #iterations per restart
    norm='inf', #->l2 ->l1 most restrictive
    eps = 0.05,
    verbose=False,
)
attack = utils.define_attack(agent, ACG, **kwargs)

atk_steps = np.arange(ATK_START,ATK_START+ATK_LEN)

In [8]:

_, obs, adv_obs, asr = utils.eval_untargeted_attack_on_step(agent,
                                                        env,
                                                        atk=attack,
                                                        atk_steps=atk_steps,
                                                        time_steps=None,
                                                        mask=observation_masks)

100%|█████████▉| 8758/8759 [00:41<00:00, 210.93it/s]


In [10]:
df_obs = pd.DataFrame(obs, columns=cols)
df_adv_obs = pd.DataFrame(adv_obs, columns=cols)

replace the observations with adv samples where the latter were generated and successful

In [16]:
df_obs.update(df_adv_obs)

label adv sample as 0

In [27]:
y = df_adv_obs.apply(lambda row: 1 if row.isnull().any() else 0, axis='columns')

In [31]:
min_samples = 25
X = deepcopy(df_obs)
X = (
    (X - X.expanding(min_periods=min_samples).mean())/X.expanding(min_periods=min_samples).std()
     ).abs().sum(axis='columns') #cumulative z-score sums
X.name = f'cumulative_scores'

In [36]:
px.line(X, x=X.index, y='cumulative_scores', color=y)